In [1]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [2]:
sent_0 = nlp(u'Donald Trump visited at the government headquarters in France today.')
sent_1 = nlp(u'Emmanuel Jean-Michel Frédéric Macron is a French politician serving as President of France and ex officio Co-Prince of Andorra since 14 May 2017.')
sent_2 = nlp(u"He studied philosophy at Paris Nanterre University, completed a Master's of Public Affairs at Sciences Po, and graduated from the École nationale d'administration (ÉNA) in 2004.")
sent_3 = nlp(u'He worked at the Inspectorate General of Finances, and later became an investment banker at Rothschild & Cie Banque.')

In [5]:
for token in sent_0.ents:
    print(token.text,token.label_)

Donald Trump PERSON
France GPE
today DATE


In [6]:
for token in sent_1:
    print(token.text,token.ent_type_)

Emmanuel PERSON
Jean PERSON
- PERSON
Michel PERSON
Frédéric PERSON
Macron PERSON
is 
a 
French NORP
politician 
serving 
as 
President 
of 
France GPE
and 
ex 
officio 
Co 
- 
Prince 
of 
Andorra 
since 
14 DATE
May DATE
2017 DATE
. 


In [8]:
for token in sent_2.ents:
    print(token.text,token.label_)

Paris Nanterre University ORG
Sciences Po ORG
École nationale d'administration ORG
2004 DATE


In [9]:
for token in sent_3.ents:
    print(token.text,token.label_)

the Inspectorate General of Finances ORG
Rothschild & Cie Banque ORG


import plac
import random
from pathlib import Path
import spacy
# training data
TRAIN_DATA = [
('Who is Shaka Khan?', {
'entities': [(7, 17, 'PERSON')]
}),
('I like London and Berlin.', {
'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')]
})
]
We've set up our basic imports and our training examples. A friendly reminder that these
are far too few examples for any serious training to happen, and that is merely a
representative example.


@plac.annotations(
model=("Model name. Defaults to blank 'en' model.", "option", "m",
str),
output_dir=("Optional output directory", "option", "o", Path),
n_iter=("Number of training iterations", "option", "n", int))
def main(model=None, output_dir=None, n_iter=100):
"""Load the model, set up the pipeline and train the
entity recognizer."""
if model is not None:
nlp = spacy.load(model) # load existing spaCy model
print("Loaded model '%s'" % model)
else:
nlp = spacy.blank('en') # create blank Language class
print("Created blank 'en' model")



We've set up annotations for where our model will be saved, as well as the number of
iterations. Our model is loaded, and we have now created a blank model.
# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
ner = nlp.create_pipe('ner')
nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
ner = nlp.get_pipe('ner')
# add labels
for _, annotations in TRAIN_DATA:
for ent in annotations.get('entities'):
ner.add_label(ent[2])
# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes): # only train NER
optimizer = nlp.begin_training()
for itn in range(n_iter):
random.shuffle(TRAIN_DATA)
losses = {}
for text, annotations in TRAIN_DATA:
nlp.update(
[text], # batch of texts
[annotations], # batch of annotations
drop=0.5, # dropout-make it harder to memorise data
sgd=optimizer, # callable to update weights
losses=losses)
print(losses)


We've set up annotations for where our model will be saved, as well as the number of
iterations. Our model is loaded, and we have now created a blank model.
# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
ner = nlp.create_pipe('ner')
nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
ner = nlp.get_pipe('ner')
# add labels
for _, annotations in TRAIN_DATA:
for ent in annotations.get('entities'):
ner.add_label(ent[2])
# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes): # only train NER
optimizer = nlp.begin_training()
for itn in range(n_iter):
random.shuffle(TRAIN_DATA)
losses = {}
for text, annotations in TRAIN_DATA:
nlp.update(
[text], # batch of texts
[annotations], # batch of annotations
drop=0.5, # dropout-make it harder to memorise data
sgd=optimizer, # callable to update weights
losses=losses)
print(losses)




We notice here that it follows the exact same training principles as the POS-tagger. We start
by adding the ner label to the pipeline, and then disabling all the other components of the
pipe so that we only train/update the NER-tagger. The training itself is straightforward,
and the nlp.update() method abstracts everything for us, letting spaCy deal with the
actual machine learning and heavy lifting.
# test the trained model
for text, _ in TRAIN_DATA:
doc = nlp(text)
print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])
# save model to output directory
if output_dir is not None:
output_dir = Path(output_dir)
if not output_dir.exists():
output_dir.mkdir()
nlp.to_disk(output_dir)
print("Saved model to", output_dir)
# test the saved model
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
for text, _ in TRAIN_DATA:
doc = nlp2(text)
print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in
doc])
if __name__ == '__main__':
plac.call(main)
Soon after our training is done, we test our model and then save it to the directory
specified. If we run the file without any errors, we should expect the following output:
Entities [('Shaka Khan', 'PERSON')]
Tokens [('Who', '', 2), ('is', '', 2), ('Shaka', 'PERSON', 3),
('Khan', 'PERSON', 1), ('?', '', 2)]
Entities [('London', 'LOC'), ('Berlin', 'LOC')]
Tokens [('I', '', 2), ('like', '', 2), ('London', 'LOC', 3),
('and', '', 2), ('Berlin', 'LOC', 3), ('.', '', 2)]
Let's now look at adding a new class to a model. The principle remains the same here; we
load the model, disable the pipes we won't be updating, add the new label, and then loop
over the examples and update them. Again, exactly like the old example, don't expect the
trained model to do any wonders – we don't have enough training examples.

In [2]:
import plac
import random
from pathlib import Path
import spacy

# new entity label
LABEL = 'ANIMAL'

In [3]:
TRAIN_DATA = [
("Horses are too tall and they pretend to care about your feelings", {
'entities': [(0, 6, 'ANIMAL')]
}),
("Do they bite?", {
'entities': []
}),
("horses are too tall and they pretend to care about your feelings", {
'entities': [(0, 6, 'ANIMAL')]
}),
("horses pretend to care about your feelings", {
'entities': [(0, 6, 'ANIMAL')]
}),
("they pretend to care about your feelings, those horses", {
'entities': [(48, 54, 'ANIMAL')]
}),
("horses?", {
'entities': [(0, 6, 'ANIMAL')]
})
]

In [10]:
@plac.annotations(
model=("en_core_web_lg", "option", "m",
str),
new_model_name=("AnimalEntity", "option", "nm", str),
output_dir=(".", "option", "o", Path),
n_iter=(10, "option", "n", int))
def main(model=None, new_model_name='animal', output_dir=None, n_iter=20):
    """Set up the pipeline and entity recognizer, and train the new entity."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")
        # Add entity recognizer to model if it's not in the pipeline
        # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner)
        # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe('ner')
        ner.add_label(LABEL)
        if model is None:
            optimizer = nlp.begin_training()
        else:
            optimizer = nlp.entity.create_optimizer()
            other_pipes = [pipe for pipe in nlp.pipe_names if pipe!='ner']
            with nlp.disable_pipes(*other_pipes):
                for itn in range(n_iter):
                    random.shuffle(TRAIN_DATA)
                    losses = {}
                    for text,annotations in TRAIN_DATA:
                        nlp.update([text],[annotations],sgd=optimizer,drop=0.35,losses=losses)
                    print(losses)
                    
    #test the model
    test_text = "Do you like horses?"
    doc = nlp(test_text)
    print("Entities are")
    for ent in doc.ents:
        print(ent.label_,ent.text)

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta['name'] = new_model_name # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)
        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)


In [11]:
plac.call(main)

usage: ipykernel_launcher.py [-h] [-m None] [-nm animal] [-o None] [-n 20]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9004 --control=9002 --hb=9001 --Session.signature_scheme="hmac-sha256" --Session.key=b"3a6167eb-21f6-4184-9c12-54860e1b72fd" --shell=9003 --transport="tcp" --iopub=9005 --f=C:\Users\z003va8b\AppData\Local\Temp\tmp-242040NM0vgboFkRW.json


SystemExit: 2

In [12]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
nlp.create_optimizer()
nlp.update()